In [1]:
from qiskit import QuantumCircuit,QuantumRegister,ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.compiler import transpile 
from qiskit.circuit.library import MCXGate 
from functions import to_binary
from functions import MaxCut
from fqga import *

from qiskit.visualization import *
import networkx as nx

import operator
from numpy import flip,array,binary_repr,insert

import math

## Setting up Graphs and Data

In [2]:
import networkx as nx
import metis
from collections import Counter


In [876]:
def create_subgraphs(graph, max_circuit_depth):
    part_size = 2
    (edgecuts, subgraphs) = metis.part_graph(graph, nparts=part_size)
    numb_vertices = Counter(subgraphs)
    req_depth_size = max(numb_vertices.values())
    
    # Adjust partition size to meet circuit depth constraint
    while max_circuit_depth < req_depth_size:
        part_size += 1
        (edgecuts, subgraphs) = metis.part_graph(graph, nparts=part_size)
        numb_vertices = Counter(subgraphs)
        req_depth_size = max(numb_vertices.values())
    
    # Create subgraphs as networkx graphs
    partitioned_subgraphs = []
    for part_id in range(part_size):
        nodes_in_part = [node for node, partition in enumerate(subgraphs) if partition == part_id]
        subgraph = graph.subgraph(nodes_in_part).copy()  # Create a subgraph and ensure it's a separate object
        partitioned_subgraphs.append(subgraph)
    
    return edgecuts, partitioned_subgraphs

In [856]:
G = nx.complete_graph(80)
print("Original Graph:", G)


# View the partition results
edgecuts, subgraphs = create_subgraphs(G, 8)
print(edgecuts, subgraphs)

numb_part = len(subgraphs)
for k in range(numb_part):
    print(f"Partition {k+1}:", subgraphs[k])

Original Graph: Graph with 80 nodes and 3160 edges
2880 [<networkx.classes.graph.Graph object at 0x7d121668d8e0>, <networkx.classes.graph.Graph object at 0x7d1217f9af00>, <networkx.classes.graph.Graph object at 0x7d1251d89790>, <networkx.classes.graph.Graph object at 0x7d1251d8a7e0>, <networkx.classes.graph.Graph object at 0x7d12154ec890>, <networkx.classes.graph.Graph object at 0x7d1214cfe480>, <networkx.classes.graph.Graph object at 0x7d121474bef0>, <networkx.classes.graph.Graph object at 0x7d1251d8a390>, <networkx.classes.graph.Graph object at 0x7d12166d4b60>, <networkx.classes.graph.Graph object at 0x7d1215864890>]
Partition 1: Graph with 8 nodes and 28 edges
Partition 2: Graph with 8 nodes and 28 edges
Partition 3: Graph with 8 nodes and 28 edges
Partition 4: Graph with 8 nodes and 28 edges
Partition 5: Graph with 8 nodes and 28 edges
Partition 6: Graph with 8 nodes and 28 edges
Partition 7: Graph with 8 nodes and 28 edges
Partition 8: Graph with 8 nodes and 28 edges
Partition 9: 

In [900]:
def evaluate(graph, part):
    binary_representation = list(part)
    binary_list = [int(bit) for bit in binary_representation]
    
    # Create two sets based on the binary representation
    set1 = {node for node, value in zip(graph.nodes(), binary_list) if value == 0}
    set2 = {node for node, value in zip(graph.nodes(), binary_list) if value == 1}
    
    # Calculate the Max-Cut fitness by counting edges between set1 and set2
    maxcut_fitness = sum(graph[u][v].get('weight', None) for u in set1 for v in set2 if graph.has_edge(u, v))
    
    return maxcut_fitness

In [899]:
part_list = [] 
for subgraph in subgraphs:
    graph = subgraph
    model = FragmentedQGA(graph, shots=1000)
    results = model.run()


    best_cut = 0
    best_str = ''
    total = 0
    for group_id, binary_list in results:
        
    
        for binary_value, other_info in binary_list:
            # You can now evaluate the binary number, for example:
            # Convert the binary string to an integer:
            binary_int = evaluate(graph, binary_value)
            total += 1
            if binary_int > best_cut:
                best_cut = binary_int
                best_str = binary_value
    print(f"Group {group_id}: {best_cut}")       
    part_list.append([best_cut, best_str])
print(part_list)

Number of vertices:8
Number of edges:28
Number of Grover Iterations:4
Population Size in Superposition:256
Iteration #1
Preparing quantum registers and creating quantum circuit...
Setting Individuals in Superposition...
Setting up Circuits...
Starting simulator...
('01001000', 9)
('11010111', 11)
('00001101', 10)
('01101110', 12)
('11001110', 11)
('10111000', 12)
('11110101', 11)
('11011110', 11)
('10101010', 11)
('10100010', 12)
Iteration #2
Preparing quantum registers and creating quantum circuit...
Setting Individuals in Superposition...
Setting up Circuits...
Starting simulator...
('10101000', 11)
('10000100', 10)
('11010111', 11)
('01101100', 10)
('00111111', 10)
('00010101', 11)
('01010001', 12)
('01011101', 10)
('11100111', 11)
('11110110', 13)
Iteration #3
Preparing quantum registers and creating quantum circuit...
Setting Individuals in Superposition...
Setting up Circuits...
Starting simulator...
('01111001', 10)
('01101010', 10)
('01000110', 10)
('11111111', 10)
('01111011',

## FragmentedQGA Model

In [898]:
part_list

[[9, '111000'], [12, '1000110'], [12, '1001010']]

## Statistical Analysis

In [860]:
def evaluate_s(part):
    binary_representation = list(part)
    binary_list = [int(bit) for bit in binary_representation]
    set1 = {node for node, value in zip(graph.nodes(), binary_list) if value == 0}
    set2 = {node for node, value in zip(graph.nodes(), binary_list) if value == 1}
    maxcut_fitness = sum(1 for u in set1 for v in set2 if graph.has_edge(u, v))
    return maxcut_fitness

In [861]:
best_cut = 0
best_par = str
total = 0
for group_id, binary_list in results:
    print(f"Group {group_id}:")
    
    for binary_value, other_info in binary_list:
        # You can now evaluate the binary number, for example:
        # Convert the binary string to an integer:
        binary_int = evaluate_s(binary_value)
        total += 1
        if binary_int > best_cut:
            best_cut = binary_int
            best_str = binary_value
    
        
        # Now you can do whatever evaluation or operation you need with the integer
        print(f"Partition: {binary_value} => Cut Value: {binary_int}, Measures: {other_info}")
    print()

Group 1:
Partition: 10101001 => Cut Value: 16, Measures: 10
Partition: 10100110 => Cut Value: 16, Measures: 10
Partition: 11111001 => Cut Value: 12, Measures: 9
Partition: 11110111 => Cut Value: 7, Measures: 10
Partition: 10111011 => Cut Value: 12, Measures: 11
Partition: 00101001 => Cut Value: 15, Measures: 10
Partition: 10111110 => Cut Value: 12, Measures: 12
Partition: 11110010 => Cut Value: 15, Measures: 9
Partition: 00011000 => Cut Value: 12, Measures: 12
Partition: 00101110 => Cut Value: 16, Measures: 10

Group 2:
Partition: 10000000 => Cut Value: 7, Measures: 11
Partition: 11111100 => Cut Value: 12, Measures: 11
Partition: 01001000 => Cut Value: 12, Measures: 11
Partition: 10010101 => Cut Value: 16, Measures: 12
Partition: 00110011 => Cut Value: 16, Measures: 11
Partition: 00001001 => Cut Value: 12, Measures: 9
Partition: 10000110 => Cut Value: 15, Measures: 11
Partition: 11011010 => Cut Value: 15, Measures: 10
Partition: 01001001 => Cut Value: 15, Measures: 10
Partition: 110001

In [862]:
best_cut

16

In [903]:
nx.set_edge_attributes(G, 1, "weight")

In [904]:
import networkx as nx

def graph_merge(graph, subgraphs, bipartitions):
    """
    Merges subgraphs into a single graph, reducing each subgraph to two vertices
    based on bipartitions and preserving all edges, with edge weights passed in 
    the bipartition list.

    Parameters:
    - graph: The original NetworkX graph.
    - subgraphs: A list of lists, each containing the nodes of a subgraph.
    - bipartitions: A list of lists, each containing a weight and a binary string representing
                     the bipartition of the subgraph.

    Returns:
    - A new merged NetworkX graph.
    """
    final_graph = nx.Graph()
    contraction_list = []
    partition_list = []
    

    # Reduce each subgraph into two vertices
    for i, (subgraph_nodes, bipartition_info) in enumerate(zip(subgraphs, bipartitions)):
        cut_weight, bipartition = bipartition_info
        cut_weight = evaluate(subgraphs[i], bipartition)
        
        # Ensure subgraph is a subgraph of graph
        subgraph_nodes = set(subgraphs[i].nodes)
        if not subgraph_nodes.issubset(graph.nodes):
            raise ValueError("subgraph must be a subgraph of graph")
    
        # Create final_graph with two initial vertices and an edge
        contraction = nx.Graph()
        contraction_vertices = [list(subgraphs[i].nodes)[0], list(subgraphs[i].nodes)[-1]]
        contraction.add_edge(contraction_vertices[0], contraction_vertices[-1], weight=cut_weight)  # Original vertices in final_graph
        if graph.has_edge(contraction_vertices[0], contraction_vertices[-1]):  # Check if the edge exists
            graph.remove_edge(contraction_vertices[0], contraction_vertices[-1])  # Remove the edge
    
        # Map subgraph nodes to their position in the bipartitions string
        list0, list1 = [],[]
        part = list(bipartitions[i][1])
        subgraph_nodes_ordered = list(subgraph_nodes)
        if len(part) != len(subgraph_nodes_ordered):
            raise ValueError("bipartitions length must match the number of vertices in subgraph")
        for i, vertex in enumerate(subgraph_nodes_ordered):
            if int(part[i]) == 0:
                list0.append(vertex)
            else:
                list1.append(vertex)
        contraction_list.append(contraction)
        partition_list.append((list0,list1))
        final_graph = nx.compose(final_graph, contraction)

    for edge in graph.edges:
        indexes = [int,int]
        for idx, subgraph in enumerate(subgraphs):
        # Check if vertex0 or vertex1 belong to the current subgraph
            if edge[0] in subgraph.nodes:
                indexes[0] = idx
            if edge[1] in subgraph.nodes:
                indexes[1] = idx
        for i in range(2):
            if edge[0] in partition_list[indexes[0]][i]:
                v0_belongs_to = i
            if edge[1] in partition_list[indexes[1]][i]:
                v1_belongs_to = i
                
        vertex0 = list(contraction_list[indexes[0]].nodes)[v0_belongs_to]
        vertex1 = list(contraction_list[indexes[1]].nodes)[v1_belongs_to]

        if indexes[0] != indexes[1]:
            if final_graph.has_edge(vertex0, vertex1):
                if 'weight' not in final_graph[vertex0][vertex1]:
                    final_graph[vertex0][vertex1]['weight'] = graph[vertex0][vertex1].get('weight', None)
                else:
                    final_graph[vertex0][vertex1]['weight'] += graph[vertex0][vertex1].get('weight', None)
            else:
                final_graph.add_edge(vertex0, vertex1, weight = graph[vertex0][vertex1].get('weight', None))

    return final_graph


In [905]:
new_graph = graph_merge(G, subgraphs, part_list)

In [906]:
print(new_graph.number_of_edges())
print(new_graph.number_of_nodes())

190
20


In [907]:
print(G.number_of_edges())

3150


In [908]:
weight_check(new_graph)

In [909]:
#for u, v, data in new_graph.edges(data=True):
#    print(f"Edge: ({u}, {v}), Weight: {data['weight']}")

In [910]:
total_weight = sum(data['weight'] for u, v, data in new_graph.edges(data=True))
print(f"Sum of weighted edges in the merged graph: {total_weight}")

Sum of weighted edges in the merged graph: 3040


In [911]:
node_mapping = {old: idx for idx, old in enumerate(new_graph.nodes())}
new_graph = nx.relabel_nodes(new_graph, node_mapping)

In [912]:
new_graph.nodes()

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19))

In [913]:
subgraphs2[0].edges(data=True)

EdgeDataView([(0, 1, {'weight': 16}), (0, 3, {'weight': 16}), (0, 2, {'weight': 16}), (0, 17, {'weight': 16}), (0, 19, {'weight': 16}), (1, 3, {'weight': 16}), (1, 2, {'weight': 16}), (1, 17, {'weight': 16}), (1, 19, {'weight': 16}), (2, 3, {'weight': 16}), (2, 17, {'weight': 16}), (2, 19, {'weight': 16}), (3, 17, {'weight': 16}), (3, 19, {'weight': 16}), (17, 19, {'weight': 16})])

In [914]:
edgecuts, subgraphs2 = create_subgraphs(new_graph, 8)
print(edgecuts, subgraphs2)

numb_part = len(subgraphs2)
for k in range(numb_part):
    print(f"Partition {k+1}:", subgraphs2[k])

133 [<networkx.classes.graph.Graph object at 0x7d125026e9f0>, <networkx.classes.graph.Graph object at 0x7d1250914530>, <networkx.classes.graph.Graph object at 0x7d121c373320>]
Partition 1: Graph with 6 nodes and 15 edges
Partition 2: Graph with 7 nodes and 21 edges
Partition 3: Graph with 7 nodes and 21 edges


In [915]:
part_list = []
for subgraph in subgraphs2:
    graph = subgraph
    model = FragmentedQGA(graph, shots=1000)
    results = model.run()


    best_cut = 0
    best_str = ''
    total = 0
    for group_id, binary_list in results:
        
    
        for binary_value, other_info in binary_list:
            # You can now evaluate the binary number, for example:
            # Convert the binary string to an integer:
            binary_int = evaluate(graph, binary_value)
            total += 1
            if binary_int > best_cut:
                best_cut = binary_int
                best_str = binary_value
    print(f"Group {group_id}: {best_cut}")       
    part_list.append([best_cut, best_str])
print(part_list)

Number of vertices:6
Number of edges:15
Number of Grover Iterations:2
Population Size in Superposition:64
Iteration #1
Preparing quantum registers and creating quantum circuit...
Setting Individuals in Superposition...
Setting up Circuits...
Starting simulator...
('111010', 27)
('001010', 26)
('010011', 24)
('010100', 24)
('110010', 26)
('100000', 27)
('001000', 23)
('101011', 25)
('100101', 23)
('100010', 29)
Iteration #2
Preparing quantum registers and creating quantum circuit...
Setting Individuals in Superposition...
Setting up Circuits...
Starting simulator...
('000111', 24)
('000111', 27)
('001010', 27)
('011111', 25)
('011110', 25)
('100101', 28)
('000000', 24)
('111011', 23)
('011011', 26)
('000011', 25)
Group 2: 144
Number of vertices:7
Number of edges:21
Number of Grover Iterations:4
Population Size in Superposition:128
Iteration #1
Preparing quantum registers and creating quantum circuit...
Setting Individuals in Superposition...
Setting up Circuits...
Starting simulator...


In [916]:
last_graph = graph_merge(new_graph, subgraphs2, part_list)

In [917]:
last_graph.edges(data=True)

EdgeDataView([(0, 19, {'weight': 144}), (0, 11, {'weight': 192}), (0, 4, {'weight': 144}), (0, 9, {'weight': 192}), (0, 18, {'weight': 144}), (19, 11, {'weight': 192}), (19, 4, {'weight': 144}), (19, 9, {'weight': 192}), (19, 18, {'weight': 144}), (4, 11, {'weight': 192}), (4, 9, {'weight': 192}), (4, 18, {'weight': 144}), (11, 9, {'weight': 256}), (11, 18, {'weight': 192}), (9, 18, {'weight': 192})])

In [918]:
total_weight = sum(data['weight'] for u, v, data in last_graph.edges(data=True))
print(f"Sum of weighted edges in the merged graph: {total_weight}")

Sum of weighted edges in the merged graph: 2656


In [919]:
graph = last_graph
model = FragmentedQGA(graph, shots=1000)
results = model.run()


best_cut = 0
best_str = ''
total = 0
for group_id, binary_list in results:
        
    
    for binary_value, other_info in binary_list:
        # You can now evaluate the binary number, for example:
        # Convert the binary string to an integer:
        binary_int = evaluate(graph, binary_value)
        total += 1
        if binary_int > best_cut:
            best_cut = binary_int
            best_str = binary_value
print(f"Group {group_id}: {best_cut}")       
part_list.append([best_cut, best_str])
print(part_list)

Number of vertices:6
Number of edges:15
Number of Grover Iterations:2
Population Size in Superposition:64
Iteration #1
Preparing quantum registers and creating quantum circuit...
Setting Individuals in Superposition...
Setting up Circuits...
Starting simulator...
('001010', 24)
('101110', 29)
('010000', 23)
('010010', 25)
('000111', 25)
('100101', 26)
('101110', 24)
('001011', 23)
('101110', 28)
('011101', 29)
Iteration #2
Preparing quantum registers and creating quantum circuit...
Setting Individuals in Superposition...
Setting up Circuits...
Starting simulator...
('111101', 28)
('110000', 27)
('011011', 24)
('000111', 26)
('011111', 24)
('101010', 24)
('011111', 27)
('110100', 24)
('111011', 26)
('101101', 27)
Group 2: 1600
[[144, '010011'], [192, '0110110'], [192, '1000011'], [1600, '100101']]


In [920]:
best_cut

1600

# Complexity Analysis


In [768]:

def f(n):
    m = math.ceil(math.log2((n*(n-1)/2)))
    return n + 2*(m + 2**math.sqrt(m - 1)) + 3

In [769]:
f(8)

29.0

In [770]:
k8 = nx.complete_graph(8)
k8.number_of_edges()

28

In [771]:
model = FragmentedQGA(k8, shots=1000)
results = model.run()

Number of vertices:8
Number of edges:28
Number of Grover Iterations:4
Population Size in Superposition:256
Iteration #1
Preparing quantum registers and creating quantum circuit...
Setting Individuals in Superposition...
Setting up Circuits...
Starting simulator...
('00101011', 10)
('00100101', 12)
('10110011', 10)
('01110001', 11)
('01100000', 11)
('01111000', 10)
('11100011', 11)
('10010011', 10)
('00110000', 9)
('10101000', 10)
Iteration #2
Preparing quantum registers and creating quantum circuit...
Setting Individuals in Superposition...
Setting up Circuits...
Starting simulator...
('01011000', 11)
('10100111', 10)
('11011111', 11)
('10010011', 9)
('11011000', 14)
('10010001', 10)
('01101111', 11)
('01011000', 10)
('00000010', 11)
('00101010', 10)
Iteration #3
Preparing quantum registers and creating quantum circuit...
Setting Individuals in Superposition...
Setting up Circuits...
Starting simulator...
('10001111', 13)
('11001101', 9)
('11010111', 10)
('01011010', 10)
('11001010', 1

In [772]:
best_cut = 0
best_par = str
total = 0
for group_id, binary_list in results:
    print(f"Group {group_id}:")
    
    for binary_value, other_info in binary_list:
        # You can now evaluate the binary number, for example:
        # Convert the binary string to an integer:
        binary_int = evaluate(k8, binary_value)
        total += 1
        if binary_int > best_cut:
            best_cut = binary_int
            best_str = binary_value
    
        
        # Now you can do whatever evaluation or operation you need with the integer
        print(f"Partition: {binary_value} => Cut Value: {binary_int}, Measures: {other_info}")
    print()

Group 1:
Partition: 00101011 => Cut Value: 16, Measures: 10
Partition: 00100101 => Cut Value: 15, Measures: 12
Partition: 10110011 => Cut Value: 15, Measures: 10
Partition: 01110001 => Cut Value: 16, Measures: 11
Partition: 01100000 => Cut Value: 12, Measures: 11
Partition: 01111000 => Cut Value: 16, Measures: 10
Partition: 11100011 => Cut Value: 15, Measures: 11
Partition: 10010011 => Cut Value: 16, Measures: 10
Partition: 00110000 => Cut Value: 12, Measures: 9
Partition: 10101000 => Cut Value: 15, Measures: 10

Group 2:
Partition: 01011000 => Cut Value: 15, Measures: 11
Partition: 10100111 => Cut Value: 15, Measures: 10
Partition: 11011111 => Cut Value: 7, Measures: 11
Partition: 10010011 => Cut Value: 16, Measures: 9
Partition: 11011000 => Cut Value: 16, Measures: 14
Partition: 10010001 => Cut Value: 15, Measures: 10
Partition: 01101111 => Cut Value: 12, Measures: 11
Partition: 01011000 => Cut Value: 15, Measures: 10
Partition: 00000010 => Cut Value: 7, Measures: 11
Partition: 00101